# The ***GroupBy*** Pattern

We have seen the ***GroupBy*** operator in ***Pandas***, but  this is actually a more general ***design pattern*** that can be utilized in many data analyics frameworks and data access interfaces, e.g. in ***SQL***. 

## GroupBy: general Pattern
<img SRC="IMG/groupby.jpg">

#### GroupBy in SQL:
```
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;
```

#### GroupBy in MongoDB
```
db.BusinessProcess.aggregate({
    "$group": {
        _id: {
            status: "$status",
            type: "$type"
        },
        count: {
            $sum: 1
        }
    }
   })
```


In [10]:
#setup example
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.211697,1.245003
1,a,two,0.650449,-0.811484
2,b,one,-0.265308,0.595631
3,b,two,-0.186973,1.366014
4,a,one,-0.099783,-0.530928


In [11]:
#group by key1 
grouped = df.groupby(df['key1'])
grouped #this is now a more complex group object

In [12]:
#and generates a table per group
for name, group in grouped:
    print ("name:", name, "\n",group)

name: a 
   key1 key2     data1     data2
0    a  one -0.211697  1.245003
1    a  two  0.650449 -0.811484
4    a  one -0.099783 -0.530928
name: b 
   key1 key2     data1     data2
2    b  one -0.265308  0.595631
3    b  two -0.186973  1.366014


In [13]:
#access group table
grouped.get_group('b')

,key1,key2,data1,data2
2,b,one,-0.265308,0.595631
3,b,two,-0.186973,1.366014


In [14]:
#get numper of entries (rows) per group
grouped.size()

key1
a    3
b    2
dtype: int64

In [15]:
#get number of group entries by columns
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


#### Think of  grouped DataFrames as 3d objects:

In [16]:
#accessing the "3d" group tables
grouped['data2'].get_group('a')

0    1.245003
1   -0.811484
4   -0.530928
Name: data2, dtype: float64

In [17]:
grouped.get_group('a')['data2']

0    1.245003
1   -0.811484
4   -0.530928
Name: data2, dtype: float64

### Group by external keys

In [18]:
#define external key years as numpy array
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([years]).mean()


2005    0.083927
2006   -0.182546
Name: data1, dtype: float64

### Group by functions

In [22]:
#sort by column and retun top n
def top(df, n=5, column='data1'):
    return df.sort_values(by=column)[-n:]

df.groupby(df['key1']).apply(top,n=5)

key1 key2     data1     data2
key1                                
a    0    a  one -0.211697  1.245003
     4    a  one -0.099783 -0.530928
     1    a  two  0.650449 -0.811484
b    2    b  one -0.265308  0.595631
     3    b  two -0.186973  1.366014

### Group-wise aggregation (apply)
<img SRC="IMG/groupby.jpg">

#### Typical build in aggregation functions:
* sum
* mean
* max / min
* quantile
* ...

In [23]:
#aggregate over the groups
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


In [24]:
grouped.sum()

,data1,data2
key1,,
a,0.338970,-0.097409
b,-0.452281,1.961645


#### Custom Aggregation Functions

In [25]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.862146,2.056487
b,0.078335,0.770382


#### Multiple aggregations

In [27]:
#just call a list of function
grouped.agg([peak_to_peak, 'mean', 'median'])

data1                            data2                    
     peak_to_peak      mean    median peak_to_peak      mean    median
key1                                                                  
a        0.862146  0.112990 -0.099783     2.056487 -0.032470 -0.530928
b        0.078335 -0.226141 -0.226141     0.770382  0.980823  0.980823

#### Suppressing the Group Keys 

In [28]:
df.groupby(df['key1']).apply(top,n=2)

key1 key2     data1     data2
key1                                
a    4    a  one -0.099783 -0.530928
     1    a  two  0.650449 -0.811484
b    2    b  one -0.265308  0.595631
     3    b  two -0.186973  1.366014

In [29]:
df.groupby(df['key1'], group_keys=False).apply(top,n=2)

,key1,key2,data1,data2
4,a,one,-0.099783,-0.530928
1,a,two,0.650449,-0.811484
2,b,one,-0.265308,0.595631
3,b,two,-0.186973,1.366014


## More  Exercises in the Lab session... 